### Manipulation of PeaskDB de novo-assisted database search results of Trocas 7 (high water, April 2019) incubation samples LC-MS/MS data using python.

Starting with:

    PeaksDB search results (.csv) of PTM-optimized database searches
    These were all searched with 15 ppm precursor tolerance and 0.5 ppm fragement ion tolerance
    Exported at <1.0% FDR
    
Goal:

    Files with stripped (no PTMs) peptide lists and
    Columns with #'s of each modification in every sequence
    Column with stripped peptide lengths (# amino acids)
    
### To use for a different file:

#### 1. Change the input file name in *IN 4*
#### 2. Use 'find + replace' (Esc + F) to replace the TROCAS # (e.g., 101) for another
#### 3. Update the NAAF factor calculated in *IN 6* into *IN 7*

In [1]:
cd /home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/T7-INC-UWPR_Apr2021_Fus_T00T24/DN50_10lgP15/202_TROCAS7_Fusion_Apr2021_PEAKS_150/

/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/T7-INC-UWPR_Apr2021_Fus_T00T24/DN50_10lgP15/202_TROCAS7_Fusion_Apr2021_PEAKS_150


In [2]:
# LIBRARIES
#import pandas library for working with tabular data
import os
os.getcwd()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import kde
#import regular expresson (regex)
import re
#check pandas version
pd.__version__

'1.0.5'

In [3]:
ls 

 202_TROCAS7_Fusion_Apr2021_PEAKS_150_peptide.csv
 202_TROCAS7_Fusion_Apr2021_PEAKS_150_protein.csv
 202_TROCAS7_Fusion_Apr2021_PEAKS_150_protein.fasta
'DB search psm.csv'
'de novo only peptides.csv'
 peptides.pep.xml
 protein-peptides.csv


In [4]:
# read the CSV into a dataframe using the pandas read_csv function
peaksdbdup202 = pd.read_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/T7-INC-UWPR_Apr2021_Fus_T00T24/DN50_10lgP15/202_TROCAS7_Fusion_Apr2021_PEAKS_150/202_TROCAS7_Fusion_Apr2021_PEAKS_150_peptide.csv")

# remove redundant rows
peaksdb202 = pd.DataFrame.drop_duplicates(peaksdbdup202)

print(peaksdb202.columns)

columns = ['Peptide', '-10lgP', 'Mass', 'Length', 'ppm', 'm/z', 'RT',
       'Area', 'Fraction', 'Scan', 'Source_File',
       '#Spec', '#Spec', 'Accession', 'PTM',
       'AScore']

peaksdb202.columns = columns

#remmove # spec and accession columns because they mess parsing up

del peaksdb202['#Spec']
del peaksdb202['Accession']
del peaksdb202['PTM']
del peaksdb202['AScore']

mean_length = peaksdb202['Length'].mean()
print(mean_length)

print("# redundant peaksdb peptides in combined dataframe", len(peaksdbdup202))
print("# nonredundant peaksdb peptides in combined dataframe", len(peaksdb202))

#look at the dataframe
peaksdb202.head()

Index(['Peptide', '-10lgP', 'Mass', 'Length', 'ppm', 'm/z', 'RT',
       'Area Trocas7-202-Bay', 'Fraction', 'Scan', 'Source File', '#Spec',
       '#Spec Trocas7-202-Bay', 'Accession', 'PTM', 'AScore'],
      dtype='object')
11.819277108433734
# redundant peaksdb peptides in combined dataframe 83
# nonredundant peaksdb peptides in combined dataframe 83


,Peptide,-10lgP,Mass,Length,ppm,m/z,RT,Area,Fraction,Scan,Source_File
0,LGEHNIDVLEGNEQFINAAK,76.05,2210.0967,20,2.3,737.7079,93.32,12900000.0,29,16124,20210407_Trocas7_662_Bay202_DDA_120min_1.raw
1,SC(+57.02)AAAGTEC(+57.02)LISGWGNTK,71.93,1881.8350,18,0.6,941.9253,90.53,1120000.0,29,15630,20210407_Trocas7_662_Bay202_DDA_120min_1.raw
2,IITHPNFNGNTLDNDIM(+15.99)LIK,58.64,2298.1677,20,0.9,767.0639,102.21,580000.0,29,17674,20210407_Trocas7_662_Bay202_DDA_120min_1.raw
3,LGEHNIDVLEGNEQ(+.98)FINAAK,58.18,2211.0808,20,3.4,738.0367,94.60,108000.0,29,16352,20210407_Trocas7_662_Bay202_DDA_120min_1.raw
4,LGEHN(+.98)IDVLEGNEQFINAAK,53.80,2211.0808,20,2.9,738.0363,96.94,90600.0,29,16760,20210407_Trocas7_662_Bay202_DDA_120min_1.raw


In [5]:
# use a count function to enumerate the # of A's (alanines) in each peptide
peaksdb202['A'] = peaksdb202['Peptide'].str.count("A")

# use a count function to enumerate the # of C's (cysteines) in each peptide
peaksdb202['C'] = peaksdb202['Peptide'].str.count("C")

# use a count function to enumerate the # of D's (aspartic acids) in each peptide
peaksdb202['D'] = peaksdb202['Peptide'].str.count("D")

# use a count function to enumerate the # of E's (glutamic acids) in each peptide
peaksdb202['E'] = peaksdb202['Peptide'].str.count("E")

# use a count function to enumerate the # of F's (phenylalanines) in each peptide
peaksdb202['F'] = peaksdb202['Peptide'].str.count("F")

# use a count function to enumerate the # of G's (glycines) in each peptide
peaksdb202['G'] = peaksdb202['Peptide'].str.count("G")

# use a count function to enumerate the # of H's (histidines) in each peptide
peaksdb202['H'] = peaksdb202['Peptide'].str.count("H")

# use a count function to enumerate the # of I's (isoleucines) in each peptide
# in peaksdb202 output, there will be no isoleucines (they're lumped in with leucines)
peaksdb202['I'] = peaksdb202['Peptide'].str.count("I")

# use a count function to enumerate the # of K's (lysines) in each peptide
peaksdb202['K'] = peaksdb202['Peptide'].str.count("K")

# use a count function to enumerate the # of L's (leucines) in each peptide
# also these include the isoleucines
peaksdb202['L'] = peaksdb202['Peptide'].str.count("L")

# use a count function to enumerate the # of M's (methionines) in each peptide
peaksdb202['M'] = peaksdb202['Peptide'].str.count("M")

# use a count function to enumerate the # of N's (asparagines) in each peptide
peaksdb202['N'] = peaksdb202['Peptide'].str.count("N")

# use a count function to enumerate the # of P's ([prolines]) in each peptide
peaksdb202['P'] = peaksdb202['Peptide'].str.count("P")

# use a count function to enumerate the # of Q's (glutamines) in each peptide
peaksdb202['Q'] = peaksdb202['Peptide'].str.count("Q")

# use a count function to enumerate the # of R's (arginines) in each peptide
peaksdb202['R'] = peaksdb202['Peptide'].str.count("R")

# use a count function to enumerate the # of S's (serines) in each peptide
peaksdb202['S'] = peaksdb202['Peptide'].str.count("S")

# use a count function to enumerate the # of T's (threonines) in each peptide
peaksdb202['T'] = peaksdb202['Peptide'].str.count("T")

# use a count function to enumerate the # of V's (valines) in each peptide
peaksdb202['V'] = peaksdb202['Peptide'].str.count("V")

# use a count function to enumerate the # of W's (tryptophans) in each peptide
peaksdb202['W'] = peaksdb202['Peptide'].str.count("W")

# use a count function to enumerate the # of Y's (tyrosines) in each peptide
peaksdb202['Y'] = peaksdb202['Peptide'].str.count("Y")

# use a count function to enumerate the # of carbamidomethylated C's in each peptide
peaksdb202['c-carb'] = peaksdb202['Peptide'].str.count("57.02")

# use a count function to enumerate the # of oxidized M's in each peptide
peaksdb202['m-oxid'] = peaksdb202['Peptide'].apply(lambda x: x.count('M(+15.99)'))

# use a lamba function to enumerate the # of deamidated N's in each peptide
peaksdb202['n-deam'] = peaksdb202['Peptide'].apply(lambda x: x.count('N(+.98)'))

# use a lamba function to enumerate the # of deamidated Q's in each peptide
peaksdb202['q-deam'] = peaksdb202['Peptide'].apply(lambda x: x.count('Q(+.98)'))

# create a column with 'stripped' peptide sequences using strip
peaksdb202['stripped_peptide'] = peaksdb202['Peptide'].str.replace(r"\(.*\)","")

# add a column with the stripped peptide length (number of AAs)
peaksdb202['stripped_length'] = peaksdb202['stripped_peptide'].apply(len)

peaksdb202['NAAF_num.'] = peaksdb202['Area'] / peaksdb202['stripped_length']

# total the number of modifications in sequence
peaksdb202['ptm-total'] = peaksdb202['c-carb'] + peaksdb202['m-oxid'] + peaksdb202['n-deam'] + peaksdb202['q-deam']

# turn all isoleucines into leucines
# this helps later in comparing Unipept peptides to PeaksDB and Comet ones
peaksdb202['stripped_IL']= peaksdb202['stripped_peptide'].str.replace('I','L')

# write modified dataframe to new txt file
peaksdb202.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDB/202_BY_T00_GD_PDB/202_BY_T00_GD_PDB.csv")

# check out the results
peaksdb202.head()

,Peptide,-10lgP,Mass,Length,ppm,m/z,RT,Area,Fraction,Scan,...,Y,c-carb,m-oxid,n-deam,q-deam,stripped_peptide,stripped_length,NAAF_num.,ptm-total,stripped_IL
0,LGEHNIDVLEGNEQFINAAK,76.05,2210.0967,20,2.3,737.7079,93.32,12900000.0,29,16124,...,0,0,0,0,0,LGEHNIDVLEGNEQFINAAK,20,645000.000000,0,LGEHNLDVLEGNEQFLNAAK
1,SC(+57.02)AAAGTEC(+57.02)LISGWGNTK,71.93,1881.8350,18,0.6,941.9253,90.53,1120000.0,29,15630,...,0,2,0,0,0,SCLISGWGNTK,11,101818.181818,2,SCLLSGWGNTK
2,IITHPNFNGNTLDNDIM(+15.99)LIK,58.64,2298.1677,20,0.9,767.0639,102.21,580000.0,29,17674,...,0,0,1,0,0,IITHPNFNGNTLDNDIMLIK,20,29000.000000,1,LLTHPNFNGNTLDNDLMLLK
3,LGEHNIDVLEGNEQ(+.98)FINAAK,58.18,2211.0808,20,3.4,738.0367,94.60,108000.0,29,16352,...,0,0,0,0,1,LGEHNIDVLEGNEQFINAAK,20,5400.000000,1,LGEHNLDVLEGNEQFLNAAK
4,LGEHN(+.98)IDVLEGNEQFINAAK,53.80,2211.0808,20,2.9,738.0363,96.94,90600.0,29,16760,...,0,0,0,1,0,LGEHNIDVLEGNEQFINAAK,20,4530.000000,1,LGEHNLDVLEGNEQFLNAAK


In [6]:
# keep only stripped peptide I/L and NAAF
db_202 = peaksdb202[['stripped_IL', 'Area', 'NAAF_num.']]

db_202.set_index('stripped_IL')

# write modified dataframe to new txt file
db_202.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/I-L_NAAFs/202_BY_T00_GD_PDB_ILnaafs.csv")

db_202.head()

,stripped_IL,Area,NAAF_num.
0,LGEHNLDVLEGNEQFLNAAK,12900000.0,645000.000000
1,SCLLSGWGNTK,1120000.0,101818.181818
2,LLTHPNFNGNTLDNDLMLLK,580000.0,29000.000000
3,LGEHNLDVLEGNEQFLNAAK,108000.0,5400.000000
4,LGEHNLDVLEGNEQFLNAAK,90600.0,4530.000000


### Exporting txt files of stripped peptides at confidence cutoffs:

In [7]:
# keep only peptide column peptides <1% FDR (this is what we exported)
pep202moddup = peaksdb202[["Peptide"]]

# keep only the stripped peptide column <1% FDR
# this is what we'll use for UniPept input, etc
pep202dup = peaksdb202[["stripped_peptide"]]

# deduplicate both of these lists
pep202mod = pep202moddup.drop_duplicates()
pep202 = pep202dup.drop_duplicates()

# print out the #s of modified and stripped peptides, deduplicated and not

print("Total modified PeaksDB peptides in 202:", len(pep202moddup))
print("Deduplicated modified PeaksDB peptides in 202:", len(pep202mod))
print("Total stripped PeaksDB peptides in 202:", len(pep202dup))
print("Deduplicated stripped PeaksDB peptides in 202:", len(pep202))

# write altered dataframe to new txt file
# used header and index parameters to get rid of 'Peptide' header and the indexing
pep202.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDB/202_BY_T00_GD_PDB/202_BY_T00_GD_PDB_stripped_peptides.txt", header=False, index=False)

# made the text file into a FASTA 
!awk '{print ">"NR"\n"$0}' /home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDB/202_BY_T00_GD_PDB/202_BY_T00_GD_PDB_stripped_peptides.txt > \
/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDB/202_BY_T00_GD_PDB/202_BY_T00_GD_PDB_stripped_peptides.fas

# look at the stripped peptides
pep202.head()

Total modified PeaksDB peptides in 202: 83
Deduplicated modified PeaksDB peptides in 202: 83
Total stripped PeaksDB peptides in 202: 83
Deduplicated stripped PeaksDB peptides in 202: 76


,stripped_peptide
0,LGEHNIDVLEGNEQFINAAK
1,SCLISGWGNTK
2,IITHPNFNGNTLDNDIMLIK
6,SSGSSYPSLLQCLK
8,VLEGNEQFINAAK


## NAAF correction and exporting files with AA and PTM totals:

In [8]:
# made a new dataframe that contains the suMN of certain columns in the modified
# peptide dataframe above 

index = ['sample total']

data = {'A': peaksdb202['A'].sum(),
        'C': peaksdb202['C'].sum(),
        'D': peaksdb202['D'].sum(),
        'E': peaksdb202['E'].sum(),
        'F': peaksdb202['F'].sum(),
        'G': peaksdb202['G'].sum(),
        'H': peaksdb202['H'].sum(),
        'I': peaksdb202['I'].sum(),
        'K': peaksdb202['K'].sum(),
        'L': peaksdb202['L'].sum(),
        'M': peaksdb202['M'].sum(),
        'N': peaksdb202['N'].sum(),
        'P': peaksdb202['P'].sum(),
        'Q': peaksdb202['Q'].sum(),
        'R': peaksdb202['R'].sum(),
        'S': peaksdb202['S'].sum(),
        'T': peaksdb202['T'].sum(),
        'V': peaksdb202['V'].sum(),
        'W': peaksdb202['W'].sum(),
        'Y': peaksdb202['Y'].sum(),
        'c-carb': peaksdb202['c-carb'].sum(),
        'm-oxid': peaksdb202['m-oxid'].sum(),
        'n-deam': peaksdb202['n-deam'].sum(),
        'q-deam': peaksdb202['q-deam'].sum(),
        'Total area': peaksdb202['Area'].sum(),
        'Total length': peaksdb202['stripped_length'].sum()
       }

totalpeaksdb202 = pd.DataFrame(data, columns=['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', \
                                              'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', \
                                              'c-carb', 'm-oxid', 
                                              'n-deam', 'q-deam', 'Total area', 'Total length'], index=index)

# calculate percentage of C's with carb (should be 1.0)
totalpeaksdb202['% C w/ carb'] = totalpeaksdb202['c-carb'] / totalpeaksdb202['C'] 

# calculate percentage of M's that are oxidized
totalpeaksdb202['% M w/ oxid'] = totalpeaksdb202['m-oxid'] / totalpeaksdb202['M'] 

# calculate percentage of N's that are deamidated
totalpeaksdb202['% N w/ deam'] = totalpeaksdb202['n-deam'] / totalpeaksdb202['N'] 

# calculate percentage of N's that are deamidated
totalpeaksdb202['% Q w/ deam'] = totalpeaksdb202['q-deam'] / totalpeaksdb202['Q'] 

# calculate NAAF denominator for all peptides in dataset i
totalpeaksdb202['NAAF denom.'] = totalpeaksdb202['Total area'] / totalpeaksdb202['Total length']

# write modified dataframe to new txt file
totalpeaksdb202.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDB/202_BY_T00_GD_PDB/202_BY_T00_GD_PDB_totals.csv")

totalpeaksdb202.head()

,A,C,D,E,F,G,H,I,K,L,...,m-oxid,n-deam,q-deam,Total area,Total length,% C w/ carb,% M w/ oxid,% N w/ deam,% Q w/ deam,NAAF denom.
sample total,71,16,40,53,29,100,25,80,41,93,...,4,6,5,414349820.0,958,1.0,0.4,0.069767,0.151515,432515.469729


In [9]:
# use the calculated NAAF factor (in totalpeaksdb202 dataframe, above) to caluclate the NAAF 
# NAAF: normalized normalized area abundance factor

NAAF20 = 432515.469729

# we'll use -10lgP > 20 as an approximate cutoff for the <1.0% FDR which is what we actually have
peaksdb202['NAAF factor'] = (peaksdb202['NAAF_num.'])/NAAF20

# make a dataframe that contains only what we need: sequences, AAs, PTMN
peaksdb202_NAAF = peaksdb202[['stripped_peptide', 'NAAF factor', 'A', 'C', 'D', 'E', 'F', 'G', 'H', \
                              'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', \
                              'c-carb', 'm-oxid', 'n-deam', 'q-deam']].copy()

# multiply the NAAF20 factor by the AAs to normalize its abundance by peak area and peptide length

peaksdb202_NAAF['A-NAAF20'] = peaksdb202_NAAF['A'] * peaksdb202['NAAF factor']
peaksdb202_NAAF['C-NAAF20'] = peaksdb202_NAAF['C'] * peaksdb202['NAAF factor']
peaksdb202_NAAF['D-NAAF20'] = peaksdb202_NAAF['D'] * peaksdb202['NAAF factor']
peaksdb202_NAAF['E-NAAF20'] = peaksdb202_NAAF['E'] * peaksdb202['NAAF factor']
peaksdb202_NAAF['F-NAAF20'] = peaksdb202_NAAF['F'] * peaksdb202['NAAF factor']
peaksdb202_NAAF['G-NAAF20'] = peaksdb202_NAAF['G'] * peaksdb202['NAAF factor']
peaksdb202_NAAF['H-NAAF20'] = peaksdb202_NAAF['H'] * peaksdb202['NAAF factor']
peaksdb202_NAAF['I-NAAF20'] = peaksdb202_NAAF['I'] * peaksdb202['NAAF factor']
peaksdb202_NAAF['K-NAAF20'] = peaksdb202_NAAF['K'] * peaksdb202['NAAF factor']
peaksdb202_NAAF['L-NAAF20'] = peaksdb202_NAAF['L'] * peaksdb202['NAAF factor']
peaksdb202_NAAF['M-NAAF20'] = peaksdb202_NAAF['M'] * peaksdb202['NAAF factor']
peaksdb202_NAAF['N-NAAF20'] = peaksdb202_NAAF['N'] * peaksdb202['NAAF factor']
peaksdb202_NAAF['P-NAAF20'] = peaksdb202_NAAF['P'] * peaksdb202['NAAF factor']
peaksdb202_NAAF['Q-NAAF20'] = peaksdb202_NAAF['Q'] * peaksdb202['NAAF factor']
peaksdb202_NAAF['R-NAAF20'] = peaksdb202_NAAF['R'] * peaksdb202['NAAF factor']
peaksdb202_NAAF['S-NAAF20'] = peaksdb202_NAAF['S'] * peaksdb202['NAAF factor']
peaksdb202_NAAF['T-NAAF20'] = peaksdb202_NAAF['T'] * peaksdb202['NAAF factor']
peaksdb202_NAAF['V-NAAF20'] = peaksdb202_NAAF['V'] * peaksdb202['NAAF factor']
peaksdb202_NAAF['W-NAAF20'] = peaksdb202_NAAF['W'] * peaksdb202['NAAF factor']
peaksdb202_NAAF['Y-NAAF20'] = peaksdb202_NAAF['Y'] * peaksdb202['NAAF factor']

# multiply the NAAF20 factor by the PTMN normalize its abundance by peak area and peptide length

peaksdb202_NAAF['ccarb-NAAF20'] = peaksdb202_NAAF['c-carb'] * peaksdb202_NAAF['NAAF factor']
peaksdb202_NAAF['moxid-NAAF20'] = peaksdb202_NAAF['m-oxid'] * peaksdb202_NAAF['NAAF factor']
peaksdb202_NAAF['ndeam-NAAF20'] = peaksdb202_NAAF['n-deam'] * peaksdb202_NAAF['NAAF factor']
peaksdb202_NAAF['qdeam-NAAF20'] = peaksdb202_NAAF['q-deam'] * peaksdb202_NAAF['NAAF factor']


# write the dataframe to a new csv
peaksdb202_NAAF.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDB/202_BY_T00_GD_PDB/202_BY_T00_GD_PDB_naaf.csv")

peaksdb202_NAAF.head()

,stripped_peptide,NAAF factor,A,C,D,E,F,G,H,I,...,R-NAAF20,S-NAAF20,T-NAAF20,V-NAAF20,W-NAAF20,Y-NAAF20,ccarb-NAAF20,moxid-NAAF20,ndeam-NAAF20,qdeam-NAAF20
0,LGEHNIDVLEGNEQFINAAK,1.491276,2,0,1,3,1,2,1,2,...,0.0,0.000000,0.000000,1.491276,0.000000,0.0,0.000000,0.00000,0.000000,0.000000
1,SCLISGWGNTK,0.235409,3,2,0,1,0,3,0,1,...,0.0,0.470819,0.470819,0.000000,0.235409,0.0,0.470819,0.00000,0.000000,0.000000
2,IITHPNFNGNTLDNDIMLIK,0.067050,0,0,2,0,1,1,1,4,...,0.0,0.000000,0.134099,0.000000,0.000000,0.0,0.000000,0.06705,0.000000,0.000000
3,LGEHNIDVLEGNEQFINAAK,0.012485,2,0,1,3,1,2,1,2,...,0.0,0.000000,0.000000,0.012485,0.000000,0.0,0.000000,0.00000,0.000000,0.012485
4,LGEHNIDVLEGNEQFINAAK,0.010474,2,0,1,3,1,2,1,2,...,0.0,0.000000,0.000000,0.010474,0.000000,0.0,0.000000,0.00000,0.010474,0.000000


In [10]:
# made a new dataframe that contains the suMN of NAAF normalized AAs for peaksdb202 peaksdb results
# also contains the suMN of the NAAF-corrected PTMN occurances for each affected residue

index = ['sample total']

data = {'NAAF': peaksdb202_NAAF['NAAF factor'].sum(),
        'A': peaksdb202_NAAF['A-NAAF20'].sum(),
        'C': peaksdb202_NAAF['C-NAAF20'].sum(),
        'D': peaksdb202_NAAF['D-NAAF20'].sum(),
        'E': peaksdb202_NAAF['E-NAAF20'].sum(),
        'F': peaksdb202_NAAF['F-NAAF20'].sum(),
        'G': peaksdb202_NAAF['G-NAAF20'].sum(),
        'H': peaksdb202_NAAF['H-NAAF20'].sum(),
        'I': peaksdb202_NAAF['I-NAAF20'].sum(),
        'K': peaksdb202_NAAF['K-NAAF20'].sum(),
        'L': peaksdb202_NAAF['L-NAAF20'].sum(),
        'M': peaksdb202_NAAF['M-NAAF20'].sum(),
        'N': peaksdb202_NAAF['N-NAAF20'].sum(),
        'P': peaksdb202_NAAF['P-NAAF20'].sum(),
        'Q': peaksdb202_NAAF['Q-NAAF20'].sum(),
        'R': peaksdb202_NAAF['R-NAAF20'].sum(),
        'S': peaksdb202_NAAF['S-NAAF20'].sum(),
        'T': peaksdb202_NAAF['T-NAAF20'].sum(),
        'V': peaksdb202_NAAF['V-NAAF20'].sum(),
        'W': peaksdb202_NAAF['W-NAAF20'].sum(),
        'Y': peaksdb202_NAAF['Y-NAAF20'].sum(),
        'c-carb': peaksdb202_NAAF['ccarb-NAAF20'].sum(),
        'm-oxid': peaksdb202_NAAF['moxid-NAAF20'].sum(),
        'n-deam': peaksdb202_NAAF['ndeam-NAAF20'].sum(),
        'q-deam': peaksdb202_NAAF['qdeam-NAAF20'].sum()
       }

totalpeaksdb202_NAAF = pd.DataFrame(data, columns=['NAAF', 'A', 'C', 'D', 'E', 'F', \
                                                   'G', 'H', 'I','K', 'L', 'M', \
                                                   'N', 'P', 'Q', 'R', 'S', \
                                                   'T', 'V', 'W', 'Y', 'c-carb', \
                                                   'm-oxid', 'n-deam', 'q-deam'], index=index)

# calculate the NAAF-corrected % modified C, M, N, Q, K, and Rs


# calculate NAAF-corrected percentage of C's with carb (should be 1.0)
totalpeaksdb202_NAAF['% C w/ carb.'] = totalpeaksdb202_NAAF['c-carb'] / totalpeaksdb202_NAAF['C'] 

# calculate NAAF-corrected percentage of M's that are oxidized
totalpeaksdb202_NAAF['% M w/ oxid'] = totalpeaksdb202_NAAF['m-oxid'] / totalpeaksdb202_NAAF['M'] 

# calculate NAAF-corrected percentage of N's that are deamidated
totalpeaksdb202_NAAF['% N w/ deam'] = totalpeaksdb202_NAAF['n-deam'] / totalpeaksdb202_NAAF['N'] 

# calculate NAAF-corrected percentage of N's that are deamidated
totalpeaksdb202_NAAF['% Q w/ deam'] = totalpeaksdb202_NAAF['q-deam'] / totalpeaksdb202_NAAF['Q'] 

# calculate NAAF summed numerator over denominator (in above cell) for all peptides in dataset i: a check
totalpeaksdb202_NAAF['NAAF check'] = totalpeaksdb202_NAAF['NAAF'] / 432515.469729

# write modified dataframe to new txt file, same name + totals
totalpeaksdb202_NAAF.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDB/202_BY_T00_GD_PDB/202_BY_T00_GD_PDB_naaf_totals.csv")

totalpeaksdb202_NAAF.head()

,NAAF,A,C,D,E,F,G,H,I,K,...,Y,c-carb,m-oxid,n-deam,q-deam,% C w/ carb.,% M w/ oxid,% N w/ deam,% Q w/ deam,NAAF check
sample total,109.768333,79.683572,2.74464,3.357645,7.178115,2.025589,10.419051,2.750349,7.753238,3.374311,...,3.36879,2.74464,0.228887,0.404957,0.035264,1.0,0.318529,0.0129,0.010657,0.000254
